In [2]:
!pip install pynput
!pip install pyaudio

import os
import wave
import pyaudio
import random
import threading
import queue
from pynput import keyboard


# Cell 1: Xác định các nhãn dữ liệu
LABELS = [
    "bat_den", "tat_den",
    "bat_quat", "tat_quat",
    "bat_may_quay", "tat_may_quay",
    "hien_thi_nhiet_do", "hien_thi_do_am", "hien_thi_chat_luong_khong_khi"
    "1","2","3","4","5","6","7","8","9","0",
    "khong", "co", "khong_co",
    "bat", "tat",
]

UNITS = ["mươi","trăm","nghìn","triệu","tỷ"]
CONFIRMATION_WORDS = ["đúng", "không đúng", "đồng ý", "không đồng ý", "tôi đồng ý", "tôi không đồng ý"]


# Cell 2: Hàm đảm bảo thư mục tồn tại
def ensure_directory(label):
    """Tạo thư mục nếu chưa tồn tại."""
    if not os.path.exists(label):
        os.makedirs(label)


# Cell 3: Hàm tạo tên file duy nhất
def generate_unique_filename(label):
    """Sinh tên file không trùng trong thư mục của nhãn."""
    while True:
        filename = f"{label}_{random.randint(1000, 9999)}.wav"
        filepath = os.path.join(label, filename)
        if not os.path.exists(filepath):
            return filepath
        
        
# Cell 4: Hàm ghi âm
def record_audio(label, rate=44100, channels=1, chunk=1024):
    """Thu âm và lưu vào thư mục tương ứng. Nhấn Enter để dừng ghi âm."""
    ensure_directory(label)
    filepath = generate_unique_filename(label)
    
    # Tạo queue để lưu frames âm thanh
    audio_frames = queue.Queue()
    # Biến để kiểm soát việc ghi âm
    is_recording = threading.Event()
    is_recording.set()  # Bắt đầu ghi âm

    def on_press(key):
        """Callback khi có phím được nhấn"""
        if key == keyboard.Key.enter:
            is_recording.clear()  # Dừng ghi âm
            return False  # Dừng listener

    def audio_callback(in_data, frame_count, time_info, status):
        """Callback để xử lý dữ liệu âm thanh"""
        if is_recording.is_set():
            audio_frames.put(in_data)
        return (in_data, pyaudio.paContinue)

    # Khởi tạo PyAudio
    audio = pyaudio.PyAudio()
    
    # Mở stream với callback
    stream = audio.open(
        format=pyaudio.paInt16,
        channels=channels,
        rate=rate,
        input=True,
        frames_per_buffer=chunk,
        stream_callback=audio_callback
    )

    print(f"Đang thu âm: {label}")
    print("Nhấn Enter để dừng ghi âm...")

    # Bắt đầu lắng nghe phím
    with keyboard.Listener(on_press=on_press) as listener:
        stream.start_stream()
        listener.join()  # Chờ đến khi nhấn Enter

    # Dừng và đóng stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Lấy tất cả frames từ queue
    frames = []
    while not audio_frames.empty():
        frames.append(audio_frames.get())

    # Lưu file
    with wave.open(filepath, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

    print(f"Đã lưu file: {filepath}\n\n")

  Using cached pynput-1.8.1-py2.py3-none-any.whl.metadata (32 kB)
Using cached pynput-1.8.1-py2.py3-none-any.whl (91 kB)


### **Thu âm điều khiển thiết bị**

In [3]:



### Thu âm bật quạt
label = "bat_quat"
record_audio(label)

### Thu âm tắt quạt
label = "tat_quat"
record_audio(label)

### Thu âm bật máy quay
label = "bat_may_quay"
record_audio(label)

### Thu âm tắt máy quay
label = "tat_may_quay"
record_audio(label)

Đang thu âm: bat_quat
Nhấn Enter để dừng ghi âm...
Đã lưu file: bat_quat\bat_quat_1048.wav


Đang thu âm: tat_quat
Nhấn Enter để dừng ghi âm...
Đã lưu file: tat_quat\tat_quat_7538.wav


Đang thu âm: bat_may_quay
Nhấn Enter để dừng ghi âm...
Đã lưu file: bat_may_quay\bat_may_quay_1370.wav


Đang thu âm: tat_may_quay
Nhấn Enter để dừng ghi âm...
Đã lưu file: tat_may_quay\tat_may_quay_2570.wav




### **Thu âm yêu cầu hiển thị thông tin**

In [ ]:
### Thu âm hiển thị nhiệt độ
label = "hien_thi_nhiet_do"
record_audio(label)

### Thu âm hiển thị độ ẩm
label = "hien_thi_do_am"
record_audio(label)

### Thu âm hiển thị chất lượng không khí
label = "hien_thi_chat_luong_khong_khi"
record_audio(label)


### Thu âm hiển thị thời gian
label = "hien_thi_thoi_gian"
record_audio(label)


### Thu âm hiển thị ngày
label = "hien_thi_ngày"
record_audio(label)

### **Thu âm các số**

In [ ]:
### Thu âm các số
for num in range(10):
    label = str(num)
    print(f"\nThu âm số {num}")
    record_audio(label)

### Thu âm các đơn vị
for unit in UNITS:
    label = unit
    print(f"\nThu âm đơn vị: {unit}")
    record_audio(label)


### **Thu âm tắt đèn**

In [ ]:
### Thu âm các từ xác nhận
for word in CONFIRMATION_WORDS:
    print(f"\nThu âm từ '{word}'")
    record_audio(word)